In [1]:
import numpy as np

In [23]:
np.random.seed(0)
X0 = np.random.multivariate_normal(mean=[2, 2, 2], cov=[[1, 0.5, 0.2], [0.5, 1, 0.3], [0.2, 0.3, 1]], size = 7) ##Gaussian distributed X given y = 0
X1 = np.random.multivariate_normal(mean=[5, 5, 5], cov=[[1, 0.5, 0.2], [0.5, 1, 0.3], [0.2, 0.3, 1]], size = 7) ##Gaussian distributed X given y = 1

print(X0)
print("")
print(X1)

[[ 1.21952488  0.08536063  0.74918613]
 [ 0.56474659  0.94944851 -0.940868  ]
 [ 1.14122611  1.22965173  1.53819741]
 [ 2.39620462  0.93114436  1.82122062]
 [ 1.65195447  1.15396082  1.50339609]
 [ 2.26395423  2.08651144  0.59243765]
 [ 0.23974459  2.90403439  2.1707248 ]]

[[4.50657382 4.98495662 3.83233137]
 [2.62751024 2.81827337 4.79787181]
 [6.45081235 4.66726315 4.08200365]
 [4.63180216 5.39353484 4.49751206]
 [6.48831059 6.52061014 6.48048261]
 [4.3532483  4.37722419 3.26387309]
 [4.15851316 5.80249346 5.8339699 ]]


In [24]:
X = np.vstack((X0, X1))  ##Features
Y = np.hstack((np.zeros(7), np.ones(7))) ##Output classes

print(X)
print(X.shape)
print("")
print(Y)
print(Y.shape)

[[ 1.21952488  0.08536063  0.74918613]
 [ 0.56474659  0.94944851 -0.940868  ]
 [ 1.14122611  1.22965173  1.53819741]
 [ 2.39620462  0.93114436  1.82122062]
 [ 1.65195447  1.15396082  1.50339609]
 [ 2.26395423  2.08651144  0.59243765]
 [ 0.23974459  2.90403439  2.1707248 ]
 [ 4.50657382  4.98495662  3.83233137]
 [ 2.62751024  2.81827337  4.79787181]
 [ 6.45081235  4.66726315  4.08200365]
 [ 4.63180216  5.39353484  4.49751206]
 [ 6.48831059  6.52061014  6.48048261]
 [ 4.3532483   4.37722419  3.26387309]
 [ 4.15851316  5.80249346  5.8339699 ]]
(14, 3)

[0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.]
(14,)


In [25]:
phi = np.mean(Y)
phi

0.5

In [26]:
mu0 = np.mean(X[Y == 0], axis=0)
mu1 = np.mean(X[Y == 1], axis=0)

print(mu0)
print("")
print(mu1)

[1.35390793 1.3343017  1.0620421 ]

[4.74525295 4.93776511 4.68400636]


In [27]:
n = len(Y)
Sigma = np.zeros((X.shape[1], X.shape[1]))

for i in range(n):
    if Y[i] == 0:
        diff = (X[i] - mu0).reshape(-1, 1)
    else:
        diff = (X[i] - mu1).reshape(-1, 1)
    Sigma += diff @ diff.T

Sigma /= n
Sigma

array([[1.06647238, 0.38507262, 0.19653479],
       [0.38507262, 0.94904182, 0.4673911 ],
       [0.19653479, 0.4673911 , 1.01850909]])

In [28]:
def gaussian_pdf(x, mu, sigma):
    n = mu.shape[0]
    coeff = 1 / (np.sqrt(((2 * np.pi) ** n) * np.linalg.det(sigma)))
    exp_term = np.exp(-0.5 * (x - mu).T @ np.linalg.inv(sigma) @ (x - mu))
    return coeff * exp_term

In [31]:
def predict(X):
  predictions = []
  for i in range(X.shape[0]):
      ##pdf P(X | Y=0) and P(X | Y=1)
      p_x_given_y0 = gaussian_pdf(X[i], mu0, Sigma)
      p_x_given_y1 = gaussian_pdf(X[i], mu1, Sigma)

      ##Posterior probabilities P(Y=0 | X) and P(Y=1 | X) by using Bayes' rule
      p_y0_given_x = p_x_given_y0 * (1 - phi)
      p_y1_given_x = p_x_given_y1 * phi

      ##Assign label based on which posterior is higher
      if p_y1_given_x > p_y0_given_x:
          predictions.append(1)
      else:
          predictions.append(0)

  return np.array(predictions)

In [32]:
predict(X)

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1])